## Concept Drift in Regression Data

In [12]:
# import the functions needed for validate and comparsion

import numpy as np
import cupy as cp
import pandas as pd
from timeit import default_timer as timer
import os

from sklearn.model_selection import train_test_split
#mean_squared_error: mse squared=true, rmse squared=false
from sklearn.metrics import mean_squared_error
from skmultiflow.data import ConceptDriftStream, FileStream
from sklearn.linear_model import SGDRegressor
from skmultiflow.data.regression_generator import RegressionGenerator
from sklearn.preprocessing import MinMaxScaler


# using plotly for plots
import plotly.express as px
from plotly.subplots import make_subplots


In [13]:
from fires import FIRES

In [14]:
# stability measurment as proposed in "Measurment the Stability of Feature Selection"

# TODO: check for case where nothing changes


def pearson_stability_ij(arr1,arr2):
    d = len(arr1)
    k_i = np.sum(arr1)
    k_j = np.sum(arr2)

    # catch edge cases as proposed in the paper under 4.1
    if (k_i == 0 or k_i == d) and k_i != k_j :
        return 0
    elif (k_j == 0 or k_j == d) and k_i != k_j :
        return 0
    elif (k_i == 0 or k_i == d) and k_i == k_j :
        return 1
    x_hat_i = k_i / d
    x_hat_j = k_j / d
    arr1 = arr1 - x_hat_i
    arr2 = arr2 - x_hat_j
    dividend = 1/d * np.sum(arr1*arr2)
    divisor = np.sqrt(1/d*np.sum(arr1**2))*np.sqrt(1/d*np.sum(arr2**2))
    return dividend/divisor

def stability_factor(selected_ftrs):
   M = len(selected_ftrs)
   sum_stabilities = 0
   for i in range(M):
       for j in range(i+1, M):
           sum_stabilities += pearson_stability_ij(selected_ftrs[i], selected_ftrs[j])
   return 1/(M*(M-1))*sum_stabilities * 2   

In [15]:
stream = ConceptDriftStream(stream=RegressionGenerator(random_state=53, n_features=100, n_informative=25), drift_stream=RegressionGenerator(random_state=43, n_features=100, n_informative=25), position=10100, width=10)

dataset_name = "Concept_drift_10"
n_selected_ftr = 25
n_window = 100
batch_size = 10

In [27]:

stream = ConceptDriftStream(stream=RegressionGenerator(random_state=53, n_features=100, n_informative=25), drift_stream=RegressionGenerator(random_state=43, n_features=100, n_informative=25), position=10100, width=10)

dataset_name = "Concept_drift_1000"
n_selected_ftr = 25
n_window = 100
batch_size = 10

In [28]:
# prepare folder for plots
folder = "plots/regression/{}".format(dataset_name)
if not os.path.exists(folder):
    os.makedirs(folder)

export_type = "pdf" # "png", "jpeg", "webp", "pdf", "svg"

In [29]:
# use SGDRegressor as predictor
stream.restart()
predictor = SGDRegressor()
scaler = MinMaxScaler()
scaler_y = MinMaxScaler()
X, y = stream.next_sample(batch_size=100)
scaler.partial_fit(X)
scaler_y.partial_fit(y.reshape(-1,1))

X = scaler.transform(X)
y = scaler_y.transform(y.reshape(-1,1))
predictor.partial_fit(X,y)

/home/kitten/anaconda3/lib/python3.8/site-packages/skmultiflow/data/concept_drift_stream.py:153: RuntimeWarning:

overflow encountered in exp

/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



SGDRegressor()

In [30]:
fires_model = FIRES(n_total_ftr=stream.n_features,
                    target_values=None,
                    mu_init=0,
                    sigma_init=1,
                    model='regression')  

In [31]:

fires_rmse = []


fires_selected_ftrs = []
fires_stability = []

start_time_all = timer()
for i in range(2000):
    # Load a new sample
    x, y = stream.next_sample(batch_size=10)
    scaler.partial_fit(x)
    scaler_y.partial_fit(y.reshape(-1,1))
    x = scaler.transform(x)
    y = scaler_y.transform(y.reshape(-1,1))
    # Select features
    ftr_weights = fires_model.weigh_features(x,y)
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    fires_selected_ftrs.append(ftr_array)

    if len(fires_selected_ftrs) >= 10:
        stability = stability_factor(fires_selected_ftrs[-10:])
        fires_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    fires_rmse.append(mean_squared_error(y, y_pred, squared=False))


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
fires_rmse = pd.Series(fires_rmse).rolling(window=n_window).mean().iloc[n_window-1:].values
end_time_all = timer()
fires_run_time = timer() - start_time_all
print("The whole fires run took {}".format(fires_run_time))

n.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A c

In [32]:
ftr_weights

array([0.29613185, 0.24736286, 0.93222964, 0.30097747, 0.51080175,
       0.57758251, 0.70474107, 0.59932158, 0.6822197 , 0.        ,
       0.66195173, 0.6481651 , 0.22242972, 0.58672609, 0.44424572,
       0.53247751, 0.41865716, 0.3901519 , 0.59953551, 0.30708955,
       0.38484706, 0.39373732, 0.66227595, 0.84469276, 0.50403914,
       0.60622315, 0.50938903, 0.30597275, 0.47505845, 0.57989146,
       0.39454489, 0.71333165, 0.45745195, 0.50269802, 0.53189301,
       0.37131964, 0.72190797, 0.42546476, 0.44016474, 0.17344528,
       0.84258208, 0.63858756, 0.55025601, 0.48194957, 0.44434693,
       0.47133567, 0.47669379, 0.50584796, 0.1584791 , 1.        ,
       0.1382581 , 0.28931548, 0.48114318, 0.60178726, 0.64988428,
       0.37694891, 0.80528452, 0.51154969, 0.24484715, 0.26429933,
       0.32416297, 0.58281751, 0.33383861, 0.67904228, 0.61199813,
       0.46219189, 0.57557191, 0.61732921, 0.39541585, 0.39678356,
       0.41099477, 0.30913242, 0.36015165, 0.32864853, 0.43620

In [33]:
# use SGDRegressor as predictor
stream.restart()
predictor = SGDRegressor()
scaler = MinMaxScaler()
scaler_y = MinMaxScaler()
X, y = stream.next_sample(batch_size=100)
scaler.partial_fit(X)
scaler_y.partial_fit(y.reshape(-1,1))

X = scaler.transform(X)
y = scaler_y.transform(y.reshape(-1,1))
predictor.partial_fit(X,y)

/home/kitten/anaconda3/lib/python3.8/site-packages/skmultiflow/data/concept_drift_stream.py:153: RuntimeWarning:

overflow encountered in exp

/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



SGDRegressor()

In [34]:
sgdr_model = SGDRegressor(penalty="l1") #penalty could be elasticnet as well
#n_selectey_ftrs?

In [35]:
sgdr_rmse = []

sgdr_selected_ftrs = []
sgdr_stability = []

start_time_all = timer()
for i in range(2000):
    # Load a new sample
    x, y = stream.next_sample(batch_size=10)
    scaler.partial_fit(x)
    scaler_y.partial_fit(y.reshape(-1,1))
    x = scaler.transform(x)
    y = scaler_y.transform(y.reshape(-1,1))
    # Select features
    sgdr_model.partial_fit(x,y)
    ftr_weights = sgdr_model.coef_
    ftr_selection = np.argsort(ftr_weights)[::-1][:n_selected_ftr]

    # Truncate x (retain only selected features, 'remove' all others, e.g. by replacing them with 0)
    x_reduced = np.zeros(x.shape)
    x_reduced[:, ftr_selection] = x[:, ftr_selection]

    # stability test
    ftr_array = np.zeros(stream.n_features)
    ftr_array[ftr_selection] = 1
    sgdr_selected_ftrs.append(ftr_array)

    if len(sgdr_selected_ftrs) >= 10:
        stability = stability_factor(sgdr_selected_ftrs[-10:])
        sgdr_stability.append(stability)


    # Test
    y_pred = predictor.predict(x_reduced)
    
    sgdr_rmse.append(mean_squared_error(y, y_pred, squared=False))
    
    


    # Train
    predictor.partial_fit(x_reduced, y)

# Restart the FileStream
sgdr_rmse = pd.Series(sgdr_rmse).rolling(window=n_window).mean().iloc[n_window-1:].values
end_time_all = timer()
sgdr_run_time = timer() - start_time_all
print("The whole sgdr run took {}".format(sgdr_run_time))


ion.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/home/kitten/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A

In [36]:
# stability
title = "Stability on dataset {}".format(dataset_name)
col_names = ["FIRES", "SGDR"]
d = {"FIRES":fires_stability, "SGDR":sgdr_stability}
df = pd.DataFrame(d, columns=col_names)
fig = px.line(df, y = col_names, title=title, labels={"index":"batches", "value":"stability"}, color_discrete_map={'FIRES': 'red', "SGDR": "blue"})
stability_trace = fig["data"]
fig.show()

In [37]:
title = "RMSE on dataset {}".format(dataset_name)
col_names = ["FIRES", "SGDR"]
d = {"FIRES":fires_rmse, "SGDR":sgdr_rmse}
df = pd.DataFrame(d, columns=col_names)
fig = px.line(df, y = col_names, title=title, labels={"index":"batches", "value":"rmse"}, color_discrete_map={'FIRES': 'red', "SGDR": "blue"})
rmse_trace = fig["data"]
fig.show()

In [38]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1) # subplot_titles=("Stability","Accuracy", "F1-Score"),
for i in range(len(stability_trace)):
    stability_trace[i]["showlegend"] = False
    fig.add_trace(stability_trace[i], row=1, col=1)
for i in range(len(rmse_trace)):
    fig.add_trace(rmse_trace[i], row=2, col=1)
fig.update_xaxes(title_text="batches", row=2, col=1)
fig.update_yaxes(title_text="Stability", row=1, col=1)
fig.update_yaxes(title_text="RMSE", row=2, col=1)

fig.write_image("{}/{}.{}".format(folder, dataset_name, export_type))
fig.show()